In [1]:

import gfapy
import re
from gfapy.sequence import rc

gfaFile = input("Enter the address of the gfa file: ")
#Loading the graph
try:
    print('Loading the graph ...')
    myGraph = gfapy.Gfa.from_file(gfaFile)
except Exception as e:
    print("Graph not loaded successfully: " + str(e))
    import pdb
    pdb.set_trace()

Enter the address of the gfa file: 111.gfa
Loading the graph ...


In [10]:
import numpy as np
import pandas as pd
# Save the data in a dataframe
df= pd.read_csv('111Sequences.tsv', sep='\t')

In [11]:
df

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,24,884
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,37,897
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,24,884
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,24,884
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,24,884
...,...,...,...,...
380,gb|AF527798.1|+|0-785|ARO:3000879|TEM-7 Partial,0,0,0
381,gb|AY590467|+|0-729|ARO:3001110|SHV-53 Partial,0,0,0
382,gb|AY130282|+|0-764|ARO:3000980|TEM-117 Partial,0,0,0
383,gb|AY130285|+|0-785|ARO:3000981|TEM-118 Partial,0,0,0


In [12]:
# Replace any spaces in PathGraphAligner with a comma
df['Node'] = df['Path_GraphAligner'].str.replace(' ', ',')
# Replace commas with '<' and add '<' at the start of each row
df['Node'] = '<' + df['Path_GraphAligner'].str.replace(',', '<')


In [13]:
df

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,Node
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,24,884,<5151
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,37,897,<5967
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,24,884,<5151
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,24,884,<5151
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,24,884,<5151
...,...,...,...,...,...
380,gb|AF527798.1|+|0-785|ARO:3000879|TEM-7 Partial,0,0,0,<0
381,gb|AY590467|+|0-729|ARO:3001110|SHV-53 Partial,0,0,0,<0
382,gb|AY130282|+|0-764|ARO:3000980|TEM-117 Partial,0,0,0,<0
383,gb|AY130285|+|0-785|ARO:3000981|TEM-118 Partial,0,0,0,<0


In [14]:
df.dtypes

Query                 object
Path_GraphAligner     object
Start_GraphAligner     int64
End_GraphAligner       int64
Node                  object
dtype: object

In [15]:
df['Node'] = df['Node'].str.replace(r'\s+', '', regex=True)

In [16]:
df

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,Node
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,24,884,<5151
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,37,897,<5967
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,24,884,<5151
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,24,884,<5151
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,24,884,<5151
...,...,...,...,...,...
380,gb|AF527798.1|+|0-785|ARO:3000879|TEM-7 Partial,0,0,0,<0
381,gb|AY590467|+|0-729|ARO:3001110|SHV-53 Partial,0,0,0,<0
382,gb|AY130282|+|0-764|ARO:3000980|TEM-117 Partial,0,0,0,<0
383,gb|AY130285|+|0-785|ARO:3000981|TEM-118 Partial,0,0,0,<0


In [17]:
#Convert the start and end position from double to int
df['End_GraphAligner'] =df['End_GraphAligner'].astype(int)
df['Start_GraphAligner'] =df['Start_GraphAligner'].astype(int)

In [18]:

# Process paths from the DataFrame'
for index, row in df.iterrows():
    path = row['Node']
    start = row['Start_GraphAligner']
    end = row['End_GraphAligner']

    # Extract nodes and orientations
    nodeOrients = path
    orientList = ['+' if ch == '>' else '-' for ch in nodeOrients]
    nodeList = re.split('>|<', nodeOrients)[1:]

    print('Generating the sequence ...')
    sequence = ''
    counter = 1
    for node, orient in zip(nodeList, orientList):
        try:
            seq = myGraph.segment(node).sequence
            if orient == '-':
                seq = rc(seq)
            if counter == 1:
                sequence = seq[start:]
            elif counter == len(nodeList):
                sequence += seq[:end]
            else:
                sequence += seq
        except AttributeError:
            print(f"Node {node} not found in the graph.")
            break

        counter += 1

    print(sequence)
    df.at[ index, 'Sequence_GraphAligner'] = sequence


Generating the sequence ...
TGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGCCGCTGGCGGTACACGCCAGCCCGCAGCCGCTTGAGCAAATTAAACAAAGCGAAAGCCAGCTGTCGGGCCGCGTAGGCATGATAGAAATGGATCTGGCCAGCGGCCGCACGCTGACCGCCTGGCGCGCCGATGAACGCTTTCCCATGATGAGCACCTTTAAAGTAGTGCTCTGCGGCGCAGTGCTGGCGCGGGTGGATGCCGGTGACGAACAGCTGGAGCGAAAGATCCACTATCGCCAGCAGGATCTGGTGGACTACTCGCCGGTCAGCGAAAAACACCTTGCCGACGGCATGACGGTCGGCGAACTCTGCGCCGCCGCCATTACCATGAGCGATAACAGCGCCGCCAATCTGCTGCTGGCCACCGTCGGCGGCCCCGCAGGATTGACTGCCTTTTTGCGCCAGATCGGCGACAACGTCACCCGCCTTGACCGCTGGGAAACGGAACTGAATGAGGCGCTTCCCGGCGACGCCCGCGACACCACTACCCCGGCCAGCATGGCCGCGACCCTGCGCAAGCTGCTGACCAGCCAGCGTCTGAGCGCCCGTTCGCAACGGCAGCTGCTGCAGTGGATGGTGGACGATCGGGTCGCCGGACCGTTGATCCGCTCCGTGCTGCCGGCGGGCTGGTTTATCGCCGATAAGACCGGAGCTAGCAAGCGGGGTGCGCGCGGGATTGTCGCCCTGCTTGGCCCGAATAACAAAGCAGAGCGCATCGTGGTGATTTATCTGCGGGATACGCCGGCGAGCATGGCCGAGCGAAATCAGCAAATCGCCGGGATCGGCGCGGCGCTGATCGAGCACTGGCAACGCTAACCCGGCGGTGGCCGCGCGCGTTATCCGGCCCGCAGCACCTCGCAGGCGTGCCGGGCGATATGACTGGCGGCGGCATCGGAAAGATGCCGGTCGGTAATGATGGTGGTGAACC

In [19]:
df

,Query,Path_GraphAligner,Start_GraphAligner,End_GraphAligner,Node,Sequence_GraphAligner
0,gb|AB023477|+|0-861|ARO:3001082|SHV-24,5151,24,884,<5151,TGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGCC...
1,gb|AB049569|+|0-861|ARO:3000958|TEM-91,5967,37,897,<5967,GGGATTTTGGTCATGAGATTATCAAAAAGGATCTTCACCTAGATCC...
2,gb|AB302939|+|8-869|ARO:3001115|SHV-60,5151,24,884,<5151,TGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGCC...
3,gb|AB372881|+|8-869|ARO:3001160|SHV-111,5151,24,884,<5151,TGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGCC...
4,gb|AB551737|+|14-875|ARO:3001177|SHV-133,5151,24,884,<5151,TGCGTTATATTCGCCTGTGTATTATCTCCCTGTTAGCCACCCTGCC...
...,...,...,...,...,...,...
380,gb|AF527798.1|+|0-785|ARO:3000879|TEM-7 Partial,0,0,0,<0,
381,gb|AY590467|+|0-729|ARO:3001110|SHV-53 Partial,0,0,0,<0,
382,gb|AY130282|+|0-764|ARO:3000980|TEM-117 Partial,0,0,0,<0,
383,gb|AY130285|+|0-785|ARO:3000981|TEM-118 Partial,0,0,0,<0,


In [20]:
df.to_csv('111Sequencestest.tsv', sep='\t', index=False)